# Tracking Sensor Bias

We want to compute the joint posterior over sensors' biases in a 2-D tracking setting.

In [1]:
from collections import OrderedDict

import pyro
import pyro.distributions as dist

import funsor
import funsor.distributions as f_dist
import funsor.ops as ops
from funsor.interpreter import interpretation, reinterpret
from funsor.optimizer import apply_optimizer
from funsor.terms import lazy
from funsor.domains import bint, reals

Simulate some synthetic data:

In [2]:
num_sensors = 5
num_frames = 100

# simulate biased sensors
sensors  = []
for _ in range(num_sensors):
    true_pos = 10 * torch.rand(2)  # in a box
    biased_pos = true_pos + torch.randn(2)
    sensors.append({
        "true_pos": true_pos,
        "biased_pos": biased_pos,
    })

# simulate a single track
track = []
z = 10 * torch.rand(2)  # initial state
for t in range(num_frames):
    # Advance latent state.
    z += torch.randn(2)
    z.clamp_(min=0, max=10)  # keep in the box
    
    # Observe via a random sensor.
    sensor_id = pyro.sample('id', dist.Categorical(torch.ones(num_sensors)))
    x = z - sensors[sensor_id]["true_pos"] + torch.randn(2)
    track.append({"sensor_id": sensor_id, "x": x})

Now let's set up a tracking problem in Funsor. We start by modeling the biases of each sensor.

In [3]:
%pdb on

Automatic pdb calling has been turned ON


In [9]:
bias = funsor.pyro.convert.mvn_to_funsor(
    dist.MultivariateNormal(
        torch.zeros(num_sensors, 2),
        torch.eye(2, requires_grad=True)  # This can be learned
    ),
    event_dims=("biased_pos",),
    real_inputs=OrderedDict([("biased_pos", reals(2))])
)(value="bias")

# Instead create a giant joint Gaussian:
bias = sum(
    funsor.pyro.convert.mvn_to_funsor(
        dist.MultivariateNormal(
            torch.zeros(2),
            torch.eye(2, requires_grad=True)  # This can be learned
        )
    )(value="bias_{}".format(i))
    
    for i in range(num_sensors)
)

AssertionError: 

> /Users/jpchen/Uber/funsor/funsor/gaussian.py(290)__init__()
    288                             if isinstance(d.dtype, int))
    289         if not torch._C._get_tracing_state():
--> 290             assert precision.shape == batch_shape + (dim, dim)
    291             assert info_vec.shape == batch_shape + (dim,)
    292 

ipdb> precision.shape
torch.Size([5, 2, 2])
ipdb> batch_shape
()
ipdb> exit


Set up the filter in funsor.

In [11]:
# Similar to funsor.pyro.hmm.GaussianHMM.__init__()
init_dist = f_dist.MultivariateNormal(torch.zeros(...), torch.eye(...))
trans_matrix = TODO
trans_dist = f_dist.MultivariateNormal(
    torch.zeros(2),
    torch.eye(2))
obs_matrix = torch.eye(2)
obs_dist = f_dist.MultivariateNormal(
    torch.zeros(2),
    torch.eye(2))

init = dist_to_funsor(initial_dist)(value="state")
trans = matrix_and_mvn_to_funsor(transition_matrix, transition_dist,
                                 ("time",), "state", "state(time=1)")
obs = matrix_and_mvn_to_funsor(observation_matrix, observation_dist,
                               ("time",), "state(time=1)", "value")
# Now this is the crux, we add bias to the observation
sensor_ids = funsor.torch.Tensor(
    torch.tensor([frame["sensor_id"] for frame in track]),
    OrderedDict([("sensor_id", bint(num_frames))]),
    dtype=len(sensors)
)
biased_observations = funsor.torch.Tensor(
    torch.tensor([frame["obs"] for frame in track]),
    OrderedDict([("sensor_id", bint(num_frames))])
)
bias_over_time = bias(sensor_id=sensor_ids)
obs = obs(value=TODO)

NameError: name 'TODO' is not defined

> <ipython-input-11-4a9982d9c245>(2)<module>()
      1 # Similar to funsor.pyro.hmm.GaussianHMM.__init__()
----> 2 init_dist = f_dist.MultivariateNormal(TODO)
      3 trans_matrix = TODO
      4 trans_dist = f_dist.MultivariateNormal(
      5     torch.zeros(2),

ipdb> exit


In [10]:
# Similar to funsor.pyro.hmm.GaussianHMM.log_prob()
ndims = max(len(batch_shape), value.dim() - event_dim)
value = tensor_to_funsor(value, ("time",), event_output=event_dim - 1,
                         dtype=self.dtype)

obs = self._obs(value=value)
result = self._trans + obs
result = sequential_sum_product(ops.logaddexp, ops.add,
                                result, "time", {"state": "state(time=1)"})
result += self._init
result = result.reduce(ops.logaddexp, frozenset(["state", "state(time=1)"]))

[autoreload of pyro failed: Traceback (most recent call last):
  File "/Users/jpchen/anaconda/envs/pyro3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/jpchen/anaconda/envs/pyro3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/Users/jpchen/anaconda/envs/pyro3/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/jpchen/anaconda/envs/pyro3/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/jpchen/Uber/pyro/pyro/__init__.py", line 1, in <module>
    import pyro.poutine as poutine
AttributeError: module 'pyro' has no

NameError: name 'self' is not defined

> <ipython-input-10-01c7e205a78b>(2)<module>()
      1 # Similar to funsor.pyro.hmm.GaussianHMM.log_prob()
----> 2 ndims = max(len(self.batch_shape), value.dim() - self.event_dim)
      3 value = tensor_to_funsor(value, ("time",), event_output=self.event_dim - 1,
      4                          dtype=self.dtype)
      5 

ipdb> exit


## Inference

Finally we have a result that is a joint Gaussian over the biases.
We can
1. optimize all parameters to maximize `result.reduce(obs.logaddexp)`
2. estimate the joint distribution over all bias parameters.